In [2]:
import os, sys
import shutil

In [ ]:
class Submission:
    def __init__(self,submission_id,submission_structure="./structure",submission_stdfile="./demo",submission_catalogue="./computing_directory",submission_logfile="./submission_log"):
        self.submission_structural = submission_structure  ##高通量计算的POSCAR所在目录  --->  ./structure ## 必须创建好的
        self.submission_stdfile = submission_stdfile  ##初始INCAR、控制计算参数变化调整脚本等共性文件所在目录   --->  ./demo ## 必须创建好的
        self.submission_catalogue = submission_catalogue  ##vasp的实际工作的 根目录  --->  ./computing_directory ## 可能忘记创建的
        self.submission_logfile = submission_logfile  ##计算日志所在目录   --->  ./submission_log ## 可能忘记创建的
        self.submission_id = None  ##计算的poscar实体
        if not os.path.isdir(self.submission_catalogue):
            os.mkdir(self.submission_catalogue)
        if not os.path.isdir(self.submission_logfile):
            os.mkdir(self.submission_logfile)

    # def native_cp(self,str_std,str_new):  ## 绕开复制文件可能遇到的权限问题
    #     #submission = Submission(self)
    #     shutil.copyfile(str_std, str_new)
    
    def std_process(self,posname):  ## 将demo文件夹拷入computing_directory文件夹，再将待计算结构拷入新生成的计算命名文件夹的通用过程
        submission = Submission(self)
        os.system(f'cp -r ./demo computing_directory/{posname}')
        shutil.copyfile(f"./structure/{posname}",f"./computing_directory/{posname}/POSCAR")   ## 待计算的结构命名为posname，内容是POSCAR文件

    def submit(self,submit_dir):   ##告知要提交计算的目录，提交并记录
        submission = Submission(self)
        mine = os.getcwd() # 记录当前位置
        os.chdir(mine) # 回到自己目录 确保安全
        os.chdir(submit_dir) # 进入计算目录
        os.system('bsub< submit.lsf')   ## 和提交排队系统的指令要一致,提交脚本命名为submit.lsf
        os.chdir(mine) # 回到自己目录 确保安全
        oname = "./v.log"
        with open(oname, 'a') as t:
            print(f"submit the {submit_dir}",file=t)
        #print('Submission submitted', submit_dir,file=submission.submission_logfile)

    def high_throughput(self,workpath):
        submission = Submission(self)
        for _ in os.listdir(workpath):
            submit

        



    def monitor(self,submit_dir):
        submission = Submission(self)
        os.chdir(submission.submission_logfile)



In [61]:
#读取文件
def read_file(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
    return lines

In [63]:
htm = open("./cac.log",'r')
htm.readline().split()


['aaaaaaa', '-1']

In [ ]:
nodelist = [int(c.split()[-1]) for c in htm.readlines()]
nodelist


In [ ]:

            run_node = sum(nodelist)

In [ ]:
import os, sys
import shutil
import time
submission_structure="./structure"
submission_stdfile="./demo"
submission_catalogue="./computing_directory"
submission_logfile="./submission_log"
node_number = 3

if not os.path.isdir(submission_catalogue):
    os.mkdir(submission_catalogue)
if not os.path.isdir(submission_logfile):
    os.mkdir(submission_logfile)

def std_process(posname):  ## 将demo文件夹拷入computing_directory文件夹，再将待计算结构拷入新生成的计算命名文件夹的通用过程
    os.system(f'cp -r ./demo computing_directory/{posname}')   ## 主要是INCAR和bash脚本(submit.lsf)
    shutil.copyfile(f"./structure/{posname}",f"./computing_directory/{posname}/POSCAR")   ## 待计算的结构命名为posname，内容是POSCAR文件

def submit(submit_dir):   ##告知要提交计算的目录，提交并记录
    vaspdone = 0
    mine = os.getcwd() # 记录当前位置
    os.chdir(mine) # 回到自己目录 确保安全
    os.chdir(submit_dir) # 进入计算目录
    os.system('bsub< submit.lsf')   ## 和提交排队系统的指令要一致,提交脚本命名为submit.lsf
    os.chdir(mine) # 回到自己目录 确保安全
    oname = "./cac.log"
    with open(oname, 'a') as t:
        print(f"submit the {submit_dir} stat 1",file=t)    ## 提示开始计算了 状态为1 占着节点
        #print(int(0),file=t)
        #print('Submission submitted', submit_dir,file=submission.submission_logfile)
    while abs(vaspdone)<1:
        if os.path.isfile(f'{submit_dir}/already_done'):  ## 脚本里要注意 计算完最后touch一个标记done文件
            print(f"{submit_dir} is done")
            vaspdone = 1
    with open(oname, 'a') as t:
        print(f"submit the {submit_dir} stat -1",file=t)    ## 提示计算结束了 状态为-1 不占节点


def high_throughput(submission_structure):   ##单节点 作为多节点的baseline
    errlog = "./hverr.log"
    for _ in os.listdir(submission_structure):
        try:
            std_process(_)
            submit(f'{submission_catalogue}/{_}')
        except Exception:
            with open(errlog, 'a') as t:
                print(f"{_} is not done",file=t)

def mut_high_throughput_monitor(node_number):  ##多节点
    errlog = "./mhv_err.log"
    _ = os.listdir(submission_structure)[0]
    std_process(_)
    submit(f'{submission_catalogue}/{_}')
    time.sleep(10)
    cac_list = os.listdir(submission_structure)[1:]  ##要计算的poscar的总的集合
    time.sleep(10)
    while True:
            htm = open("./cac.log",'a')
            nodelist = [int(c.split()[-1]) for c in htm.readlines()]
            run_node = sum(nodelist)
            if run_node < node_number:
                for i in (cac_list):
                    try:
                        std_process(i)
                        time.sleep(3)   ## std过程有copy文件夹的行为，所以要等待一会
                        submit(f'{submission_catalogue}/{i}')
                        cac_list.remove(i)
                    except Exception:
                        print("std_p err")
                        with open(errlog, 'a') as t:
                            print(f"{i} is not done",file=t)
            time.sleep(30)   ## 每隔30秒看一眼

mut_high_throughput_monitor(3)



In [44]:
### 测试提交脚本 命名为submit.lsf

## 使用了vaspkit作为计算辅助工具

#!/bin/bash
#BSUB -q normal
#BSUB -o out.%J.txt
#BSUB -e error.%J.txt
#BSUB -J test_hv
#BSUB -n 24
##BSUB -R "span[ptile=16]"
#BSUB -R "span[hosts=1]"

#source /share/apps/intel/ipsxe2016/parallel_studio_xe_2016.0.047/psxevars.sh >/dev/null 2>&1
source /share/apps/intel/ipsxe2015u5/parallel_studio_xe_2015/psxevars.sh >/dev/null 2>&1

echo 1
#Run MultiTheads per MPI Process
#-genv  OMP_NUM_THREADS 1
export OMP_NUM_THREADS=1
echo 2

#Run MPI over infiniband
#mpirun -genv I_MPI_FABRICS tmi -bootsyrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp

#Run MPI within one Compute Note;

##############mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result


vaspkit -task 102
wait
vaspkit -task 103
wait
cp INCAR03 INCAR
mkdir {01_relax,02_scf,03_dos,04_bands,05_becs,06_ela,07_phonopy}
mkdir 01_relax/{01_do,02_copy}
cp INCAR KPOINTS POSCAR POTCAR 01_relax/01_do/
cd 01_relax/01_do/
cp ../../INCAR01 INCAR
mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result1
wait
mv CONTCAR POSCAR
wait
cp ../../INCAR02 INCAR
mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result2
wait
mv CONTCAR POSCAR
wait
python ../../loop.py > loopdata
loop=$(cat loopdata)
i=1
while (($i<=3)) && (($loop>10))
do 
i=`expr $i + 1`
cp ../../INCAR01 INCAR
mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result1
wait
mv CONTCAR POSCAR
wait
cp ../../INCAR02 INCAR
mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result2
wait
mv CONTCAR POSCAR
wait
python ../../loop.py >> loopdata
loop=$(tail -n 1 loopdata)
echo $i > loop_indx
done
######

cp ../../INCAR03 INCAR

mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result3
wait
python ../../loop.py >> loopdata
wait

########conda activate pyxtal

cp CONTCAR INCAR KPOINTS POSCAR POTCAR  ../02_copy/
cp CONTCAR INCAR KPOINTS POSCAR POTCAR ../../02_scf/
cd ../../02_scf/
mv CONTCAR POSCAR
####sed -i "s/4/8/g" KPOINTS  ####在这里写自洽和优化不同的INCAR参数
sed -i "s/^.*\\bISTART\\b.*$/ ISTART = 0/" INCAR
sed -i "s/^.*\\bICHARG\\b.*$/ ICHARG = 2/" INCAR
sed -i "s/^.*\\bLWAVE\\b.*$/ LWAVE  = .TRUE./" INCAR
sed -i "s/^.*\\bLCHARG\\b.*$/ LCHARG = .TRUE./" INCAR
sed -i "s/^.*\\bNSW\\b.*$/ NSW    = 0/" INCAR
sed -i "s/^.*\\bIBRION\\b.*$/ IBRION = -1/" INCAR
sed -i "s/^.*\\bISIF\\b.*$/ ISIF   = 2/" INCAR
sed -i "/LCHARG/ a\ LORBIT = 11" INCAR
mpirun -genv I_MPI_FABRICS shm -bootstrap lsf /share/apps/vasp.5.3.3.intel2015u5/vasp.5.3/vasp>result
wait
touch already_done
wait
python ../ener_per.py > ../enerlog
python ../ener_per.py > ../02_scf/enerlog
wait


SyntaxError: invalid syntax (Temp/ipykernel_48924/1345663136.py, line 15)

In [6]:
int("asfdjbaof aiofhoafdho -1".split()[-1])

-1

In [43]:
a = (os.listdir(r"C:\Users\lenovo\Desktop\RWNN\RWNN"))
# 从集合中删除
a.remove('RDF.py')
for i in a:
    print(i)

formation_energy.hdf5
formation_energy.hdf5.json
gaussian_sample_0106.py
gen_dnn_pred.py
get_EPV.py
loop.py
nor.sh
pred.py
svasp.lsf
val_pred.ipynb
work.py


In [32]:
b = a..remove(set['RDF.py'])
b

TypeError: 'types.GenericAlias' object is not iterable

In [ ]:
import os, sys
import shutil
import time
submission_structure="./structure"
submission_stdfile="./demo"
submission_catalogue="./computing_directory"
#submission_logfile="./submission_log"
node_number = 3

if not os.path.isdir(submission_catalogue):
    os.mkdir(submission_catalogue)
#if not os.path.isdir(submission_logfile):
#    os.mkdir(submission_logfile)

def std_process(posname):  ## 将demo文件夹拷入computing_directory文件夹，再将待计算结构拷入新生成的计算命名文件夹的通用过程
    os.system(f'cp -r ./demo computing_directory/{posname}')   ## 主要是INCAR和bash脚本(submit.lsf)
    time.sleep(3)
    shutil.copyfile(f"./structure/{posname}",f"./computing_directory/{posname}/POSCAR")   ## 待计算的结构命名为posname，内容是POSCAR文件

def submit(submit_dir):   ##告知要提交计算的目录，提交并记录
    vaspdone = 0
    mine = os.getcwd() # 记录当前位置
    os.chdir(mine) 
    os.chdir(submit_dir) # 进入计算目录
    os.system('bsub< submit.lsf')   ## 和提交排队系统的指令要一致,提交脚本命名为submit.lsf
    os.chdir(mine) # 回到自己目录 确保安全
    oname = "./cac.log"
    with open(oname, 'a') as t:
        print(f"submit the {submit_dir} stat 1",file=t)    ## 提示开始计算了 状态为1 占着节点

def high_throughput(submission_structure):   ##单节点 作为多节点的baseline
    errlog = "./hverr.log"
    for _ in os.listdir(submission_structure):
        try:
            std_process(_)
            submit(f'{submission_catalogue}/{_}')
        except Exception:
            with open(errlog, 'a') as t:
                print(f"{_} is not done",file=t)

def get_node_info(path):
    num = 0
    all_cac = len(os.listdir(path))  ##一共提交过
    for _ in os.listdir(path):
        if os.path.isfile(f'{path}/{_}/already_done'):  ## 脚本里要注意 计算完最后touch一个标记done文件
            num += 1
        else:
            runing = "./runing.log"
            with open(runing, 'w') as run:
                print(f"{_} is not done",file=run)
    return all_cac - num

def mut_high_throughput_monitor(node_number):  ##多节点
    errlog = "./mhv_err.log"
    _ = os.listdir(submission_structure)[0]
    std_process(_)
    time.sleep(1)
    submit(f'{submission_catalogue}/{_}')
    time.sleep(1)
    cac_list = os.listdir(submission_structure)[1:]  ##要计算的poscar的总的集合
   
    while True:
            run_node = get_node_info(path = submission_catalogue)
            if run_node < node_number:
                for i in (cac_list):
                    try:
                        std_process(i)
                        submit(f'{submission_catalogue}/{i}')
                        cac_list.remove(i)
                    except Exception:
                        print("std_p err")
                        with open(errlog, 'a') as t:
                            print(f"{i} is not done",file=t)
           # time.sleep(1)   ## 每隔30秒检测一次

mut_high_throughput_monitor(5)
